In [16]:
import time
import firebase_admin 
from firebase_admin import credentials, db 

In [17]:
cred = credentials.Certificate("creds.json")
firebase_admin.initialize_app(cred, {
    "databaseURL": "https://alert-8c0c7-default-rtdb.asia-southeast1.firebasedatabase.app/"
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [18]:
ambulance = db.reference("/ambulence_current_location")
accident_locations = db.reference("/accident_locations")

In [19]:
def assign_ambulances():
    while True:
        accident_data = accident_locations.get()
        ambulance_data = ambulance.get()

        if not accident_data or not ambulance_data:
            time.sleep(5)
            continue

        for key, location in accident_data.items():
            dis = float('inf')
            assign_amb = None

            for amb_id, amb_location in ambulance_data.items():
                if amb_location['isFree']:
                    dis_temp = (amb_location['latitude'] - location['latitude'])**2 + (amb_location['longitude'] - location['longitude'])**2
                    if dis_temp < dis:
                        dis = dis_temp
                        assign_amb = amb_id

            if assign_amb:
                db.reference("/ambulence_current_location").update({f"{assign_amb}/isFree": False})
                db.reference("/ambulance_status").update({f"{assign_amb}": location['Nearest_hospital']})
                db.reference(f"/accident_locations/{key}/Assign").set(True)


        time.sleep(5)

In [20]:
assign_ambulances()

KeyboardInterrupt: 